In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50, DenseNet121
import tensorflow as tf
import psutil
from pympler import asizeof
import sklearn

# Load your CSV file
csv_file_path = '../DL_for_Hin_Chest_X_Ray/Data_Entry_2017_filtered_2.csv'
df = pd.read_csv(csv_file_path)

# Initialize constants
IMAGE_DIR = "../DL_for_Hin_Chest_X_Ray/HIN_archive/images/"

# Initialize the multi-label binarizer
mlb = MultiLabelBinarizer()

unique_labels = df["Finding Labels"].str.split("|").explode().unique()
mlb.fit([unique_labels])
labels_for_class = ['Atelectasis', 'Effusion', 'Infiltration', 'No Finding']

In [2]:
def preprocess_image(file_path, image_size):
    """
    Loads and preprocesses an image from the given file path.
    Resizes to the specified image size and normalizes pixel values.
    """
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    if image is None:
        return None
    image = cv2.resize(image, (image_size, image_size))
    # image = image / 255.0  # Normalize pixel values to [0, 1]
    return image


def prepare_data(df, image_size, image_dir=IMAGE_DIR):
    """
    Prepares images and labels from the dataset for model training.
    - Loads images based on 'Image Index' in the dataframe.
    - Converts 'Finding Labels' to one-hot encoded vectors.
    - Returns arrays of images and labels.
    """
    images = []
    labels = []
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        # Construct image path
        image_path = os.path.join(image_dir, row["Image Index"])
        image = preprocess_image(image_path, image_size)
        
        if image is not None:
            images.append(image)
            # Convert labels into a list of diseases, then one-hot encode
            label = row["Finding Labels"].split("|")
            labels.append(label)
    
    # Convert lists to arrays
    images = np.array(images).reshape(-1, image_size, image_size, 1)  # Adding channel dimension for grayscale
    labels = mlb.transform(labels)  # Convert labels to multi-label one-hot encoding
    
    return images, labels

def weighted_cross_entropy2(y_true, y_pred):
    tf.print("y_true", y_true, summarize=-1)
    tf.print("y_pred", y_pred, summarize=-1)

    
    positive_cases = tf.equal(y_true[:, -1], 0)  # Positive when last column is 0
    negative_cases = tf.equal(y_true[:, -1], 1)  # Negative when last column is 1
    positive_cases = tf.cast(positive_cases, tf.float32)
    negative_cases = tf.cast(negative_cases, tf.float32)
    
    tf.print("positive_cases", positive_cases, summarize=-1)
    tf.print("negative_cases", negative_cases, summarize=-1)
    
    count_positive = tf.reduce_sum(positive_cases)
    count_negative = tf.reduce_sum(negative_cases)
    
    tf.print("count_positive", count_positive, summarize=-1)
    tf.print("count_negative", count_negative, summarize=-1)
    
    total_samples = tf.cast(tf.shape(y_true)[0], tf.float32)
    
    tf.print("total_samples", total_samples)

    beta_p = total_samples / count_positive
    beta_n = total_samples / count_negative
    
    beta_p = tf.clip_by_value(beta_p, 1e-7, 1e7) #  Clipping 
    beta_n = tf.clip_by_value(beta_n, 1e-7, 1e7)
    
    tf.print("beta_p", beta_p, summarize=-1)
    tf.print("beta_n", beta_n, summarize=-1)
    
    
    y_pred_binary = tf.stack([tf.reduce_sum(y_pred[:, :-1], axis=-1), y_pred[:, -1]], axis=-1)
    
    y_pred_binary = tf.cast(y_pred_binary, tf.float32)

    tf.print("y_pred_binary", y_pred_binary, summarize=-1)
    
    weighted_positive_loss = beta_p * tf.reduce_sum(-positive_cases * tf.math.log(y_pred_binary[:, 0] + 1e-7))
    weighted_negative_loss = beta_n * tf.reduce_sum(-negative_cases * tf.math.log(y_pred_binary[:, 1] + 1e-7))
    
    tf.print("weighted_positive_loss", weighted_positive_loss, summarize=-1)
    tf.print("weighted_negative_loss", weighted_negative_loss, summarize=-1)   
    
    return weighted_positive_loss + weighted_negative_loss

def create_resnet_model(image_size):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3), pooling=None)
    base_model.trainable = False
    
    model = Sequential()
    model.add(base_model)
    model.add(Conv2D(2048, (1, 1), activation="relu")) # Transition Layer
    model.add(GlobalAveragePooling2D())   
    model.add(Dense(4, activation='softmax', name="predictions"))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[AUC()])
    return model

def evaluate(model, images_val, labels_val, num_classes):
    # Get the model's predictions (probabilities) for the validation set
    probabilities = model.predict(images_val)

    # Initialize an empty list to store the AUCs
    auc_per_class = {}

    # Calculate AUC for each class
    for class_idx in range(num_classes):
        true_labels = labels_val[:, class_idx]  # True labels for this class
        pred_probs = probabilities[:, class_idx]  # Predicted probabilities for this class

        # Calculate the AUC for this class
        auc = sklearn.metrics.roc_auc_score(true_labels, pred_probs)
        auc_per_class[labels_for_class[class_idx]] = auc        
        
    probabilities_transformed = probabilities.argmax(axis=1)
    labels_val_transformed = labels_val.argmax(axis=1)

    balanced_acc = sklearn.metrics.balanced_accuracy_score(labels_val_transformed, probabilities_transformed)
    acc = sklearn.metrics.accuracy_score(labels_val_transformed, probabilities_transformed)

    return auc_per_class, balanced_acc, acc


In [3]:
IMAGE_SIZE = 224
NUMBER_OF_IMAGES = 40000

def normalize_image(img, label):
    img = tf.cast(img, np.float32) / 255.0
    return img, label

images, labels = prepare_data(df[:NUMBER_OF_IMAGES], IMAGE_SIZE)
images_test, labels_test = prepare_data(df[-(int(NUMBER_OF_IMAGES / 5)):], IMAGE_SIZE)


################# only used if model requires 3 channels
images = np.repeat(images[..., np.newaxis], 3, axis=-1).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
images_test = np.repeat(images_test[..., np.newaxis], 3, axis=-1).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

images_train, images_val, labels_train, labels_val = sklearn.model_selection.train_test_split(images, labels, random_state=42, test_size=0.2)

100%|██████████| 8000/8000 [01:30<00:00, 88.53it/s] 


In [ ]:
BATCH_SIZE = 32
    
dataset = tf.data.Dataset.from_tensor_slices((images_train, labels_train))
dataset = dataset.map(normalize_image)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((images_val, labels_val))
val_dataset = val_dataset.map(normalize_image)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=False)
    
model = create_resnet_model(IMAGE_SIZE)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[AUC()])

class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(labels_train.argmax(axis=1)), y=labels_train.argmax(axis=1))
class_weights = {i: class_weights[i] for i in range(len(class_weights))}


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(dataset, epochs=20, batch_size=BATCH_SIZE, steps_per_epoch=len(images_train) // BATCH_SIZE, validation_data=val_dataset, validation_steps=len(images_val)//BATCH_SIZE, callbacks=[early_stopping], verbose=2)#, class_weight=class_weights)

evals = evaluate(model, images_test, labels_test, len(unique_labels))
print(evals)

2025-02-27 22:17:46.298654: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-02-27 22:17:46.299650: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-27 22:17:46.300968: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-27 22:17:46.301938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-27 22:17:46.302549: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/20


2025-02-27 22:18:23.607861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1000/1000 - 254s - 254ms/step - auc_1: 0.8341 - loss: 0.9116 - val_auc_1: 0.8445 - val_loss: 0.8857
Epoch 2/20
1000/1000 - 238s - 238ms/step - auc_1: 0.8406 - loss: 0.8843 - val_auc_1: 0.8483 - val_loss: 0.8823
Epoch 3/20
1000/1000 - 245s - 245ms/step - auc_1: 0.8453 - loss: 0.8784 - val_auc_1: 0.8528 - val_loss: 0.8798
Epoch 4/20
1000/1000 - 241s - 241ms/step - auc_1: 0.8500 - loss: 0.8728 - val_auc_1: 0.8572 - val_loss: 0.8723
Epoch 5/20
1000/1000 - 238s - 238ms/step - auc_1: 0.8538 - loss: 0.8674 - val_auc_1: 0.8596 - val_loss: 0.8646
Epoch 6/20
1000/1000 - 239s - 239ms/step - auc_1: 0.8569 - loss: 0.8629 - val_auc_1: 0.8622 - val_loss: 0.8609
Epoch 7/20
1000/1000 - 238s - 238ms/step - auc_1: 0.8593 - loss: 0.8585 - val_auc_1: 0.8639 - val_loss: 0.8562
Epoch 8/20
1000/1000 - 239s - 239ms/step - auc_1: 0.8607 - loss: 0.8563 - val_auc_1: 0.8651 - val_loss: 0.8534
Epoch 9/20
1000/1000 - 238s - 238ms/step - auc_1: 0.8623 - loss: 0.8538 - val_auc_1: 0.8667 - val_loss: 0.8498
Epoch 10/20


In [ ]:
plt.figure(figsize=(8, 6))

font_size = 16

plt.plot(history.history['loss'], label="Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")

plt.xlabel("Epochs", fontsize=font_size)
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.legend(fontsize=font_size, loc="center right")

plt.savefig("evals/history_256px_1000samples_model3.pdf", bbox_inches='tight', pad_inches=0)

plt.show()


In [ ]:
# Resolution

BATCH_SIZE = 16
NUMBER_OF_IMAGES = 1000
evaluations = {}

def normalize_image(img, label):
    img = tf.cast(img, np.float32) / 255.0
    return img, label


for ev_image_size in [50, 100, 150, 200, 250, 300, 350, 400]:
    
    images, labels = prepare_data(df[:NUMBER_OF_IMAGES], ev_image_size)
    images_test, labels_test = prepare_data(df[-(int(NUMBER_OF_IMAGES / 5)):], ev_image_size)

    images_train, images_val, labels_train, labels_val = sklearn.model_selection.train_test_split(images, labels, random_state=42, test_size=0.2)

    dataset = tf.data.Dataset.from_tensor_slices((images_train, labels_train))
    dataset = dataset.map(normalize_image)
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    dataset = dataset.repeat()
    
    val_dataset = tf.data.Dataset.from_tensor_slices((images_val, labels_val))
    val_dataset = val_dataset.map(normalize_image)
    val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=False)
        
    model = create_model_3(ev_image_size)
        
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(labels_train.argmax(axis=1)), y=labels_train.argmax(axis=1))
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(dataset, epochs=20, batch_size=BATCH_SIZE, steps_per_epoch=len(images_train) // BATCH_SIZE, validation_data=val_dataset, validation_steps=len(images_val)//BATCH_SIZE, callbacks=[early_stopping], class_weight=class_weights, verbose=2)
    
    evals = evaluate(model, images_test, labels_test, len(unique_labels))
    
    evaluations[ev_image_size] = evals
    print()
    print(evals)
    print()
    
print(evaluations)

In [ ]:
# Amount of Images

BATCH_SIZE = 32
NUMBER_OF_IMAGES = 40000
IMAGE_SIZE = 224

images_global, labels_global = prepare_data(df[:NUMBER_OF_IMAGES], IMAGE_SIZE)
images_global = np.repeat(images_global[..., np.newaxis], 3, axis=-1).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

In [ ]:

evaluations = {}

def normalize_image(img, label):
    img = tf.cast(img, np.float32) / 255.0
    return img, label


for ev_number_of_images in [1000, 10000, 20000, 30000, 40000]:
        
    images, labels = images_global[:ev_number_of_images], labels_global[:ev_number_of_images]
    images_test, labels_test = images_global[-(int(ev_number_of_images / 5)):], labels_global[-(int(ev_number_of_images / 5)):]

    images_train, images_val, labels_train, labels_val = sklearn.model_selection.train_test_split(images, labels, random_state=42, test_size=0.2)

    dataset = tf.data.Dataset.from_tensor_slices((images_train, labels_train))
    dataset = dataset.map(normalize_image)
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    dataset = dataset.repeat()
    
    val_dataset = tf.data.Dataset.from_tensor_slices((images_val, labels_val))
    val_dataset = val_dataset.map(normalize_image)
    val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=False)
        
    model = create_resnet_model(IMAGE_SIZE)
        
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(labels_train.argmax(axis=1)), y=labels_train.argmax(axis=1))
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(dataset, epochs=20, batch_size=BATCH_SIZE, steps_per_epoch=len(images_train) // BATCH_SIZE, validation_data=val_dataset, validation_steps=len(images_val)//BATCH_SIZE, callbacks=[early_stopping], verbose=2, class_weight=class_weights)
    
    evals = evaluate(model, images_test, labels_test, len(unique_labels))
    
    evaluations[ev_number_of_images] = evals
    print()
    print(evals)
    print()
    
print(evaluations)

In [ ]:
plt.figure(figsize=(8, 6))

with open("pickles/resolution_1000samples_nolr_model3.pkl", "wb") as file:
    pickle.dump(evaluations, file)

aucs = [evalu[0] for evalu in evaluations.values()]
aucs_at = [evalu["Atelectasis"] for evalu in aucs]
aucs_ef = [evalu["Effusion"] for evalu in aucs]
aucs_in = [evalu["Infiltration"] for evalu in aucs]
aucs_nf = [evalu["No Finding"] for evalu in aucs]
baccs = [evalu[1] for evalu in evaluations.values()]
accs = [evalu[2] for evalu in evaluations.values()]

x_values = [50, 100, 150, 200, 250, 300, 350, 400]

font_size = 16

# plt.plot(x_values, accs, label="Accuracy")
# plt.plot(x_values, baccs, label="Balanced Accuracy")

plt.plot(x_values, aucs_at, label="Atelectasis")
plt.plot(x_values, aucs_ef, label="Effusion")
plt.plot(x_values, aucs_in, label="Infiltration")
plt.plot(x_values, aucs_nf, label="No Finding")

plt.xlabel("Resolution in pixels x pixels", fontsize=font_size)
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.legend(fontsize=font_size, loc="lower right")

#plt.savefig("evals/amount_resnet.pdf", bbox_inches='tight', pad_inches=0)

plt.show()

In [ ]:
# Baseline

images, labels = prepare_data(df[:10000], 1)

# Calculate class probabilities
class_counts = np.sum(labels, axis=0)
class_probabilities = class_counts / len(labels)

# Prepare the test data
images_test, labels_test = prepare_data(df[-2000:], 1)

# Generate predictions based on class probabilities
predictions = []
for _ in range(len(labels_test)):
    predicted_class = np.random.choice(len(class_probabilities), p=class_probabilities)
    predictions.append(predicted_class)
    
predictions_proba = np.zeros((len(labels_test), len(class_probabilities)))
for i, predicted_class in enumerate(predictions):
    predictions_proba[i, predicted_class] = 1
    
evals = []
for class_idx in range(4):
    true_labels = labels_test[:, class_idx]  # True labels for this class
    pred_probs = predictions_proba[:, class_idx]  # Predicted probabilities for this class

    # Calculate the AUC for this class
    auc = sklearn.metrics.roc_auc_score(true_labels, pred_probs)
    evals.append(auc)
    print(f"AUC for class {class_idx}: {auc}")
    
    
probabilities_transformed = predictions
labels_val_transformed = labels_test.argmax(axis=1)

balanced_acc = sklearn.metrics.balanced_accuracy_score(labels_val_transformed, probabilities_transformed)
acc = sklearn.metrics.accuracy_score(labels_val_transformed, probabilities_transformed)

print(balanced_acc)
print(acc)

In [ ]:
# tf.keras.utils.plot_model(create_model_2(256), show_shapes=False, rankdir="LR")

birne = create_model_3(256)
def myprint(s):
    with open('test.txt','w') as f:
        print(s, file=f)

birne.summary(print_fn=myprint)

In [ ]:
# Train for balanced accuracy
class_weights = {}
num_classes = len(unique_labels)

for class_idx in range(num_classes):
    class_count = np.sum(labels[:, class_idx])
    class_weight = len(labels) / (num_classes * (class_count + 1e-5)) 
    class_weights[class_idx] = class_weight


model.fit(dataset, epochs=10, batch_size=BATCH_SIZE, steps_per_epoch=len(images) // BATCH_SIZE, class_weight=class_weights, validation_data=(images_val, labels_val))
probabilities = model.predict(images_test)
predictions = probabilities.argmax(axis=1)
labels_test = labels_test.argmax(axis=1)

balanced_acc = sklearn.metrics.balanced_accuracy_score(labels_test, predictions)

balanced_acc